In [2]:
import os, math, sys
import pandas as pd
import pandapower as pp
import numpy as np

from pandapower.file_io import from_json, to_json

main_dir = os.path.abspath(os.path.join(os.path.abspath(""), ".."))
data_dir = os.path.join(main_dir, 'Modified_116_LV_CSV')

if main_dir not in sys.path:
    sys.path.insert(0, main_dir)

load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\re2.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python31

In [3]:
net48 = from_json(os.path.join(main_dir, 'json_networks', '48_loads_network.json'))
net48

This pandapower network includes the following parameter tables:
   - bus (117 elements)
   - asymmetric_load (48 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (115 elements)
   - trafo (1 element)
   - controller (1 element)
 and the following results tables:
   - res_bus_3ph (117 elements)
   - res_line_3ph (115 elements)
   - res_trafo_3ph (1 element)
   - res_ext_grid_3ph (1 element)
   - res_shunt_3ph (1 element)

In [4]:
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
loadShape_df = pd.read_csv(os.path.join(data_dir, 'LoadShapes.csv'), skiprows=1, sep=';')
loads_df.head(), loadShape_df.head()

merged_load = loads_df.merge(loadShape_df, left_on="Yearly", right_on="Name", how="left")
merged_load.head(10)

,Name_x,numPhases,Bus,phases,kV,Model,Connection,kW,PF,Yearly,Name_y,npts,minterval,File,useactual
0,LOAD1,1,5.0,A,0.23,1,wye,1,0.95,Shape_1,Shape_1,1440,1,Load_profile_1.csv,True
1,LOAD2,1,7.0,B,0.23,1,wye,1,0.95,Shape_2,Shape_2,1440,1,Load_profile_2.csv,True
2,LOAD3,1,10.0,A,0.23,1,wye,1,0.95,Shape_3,Shape_3,1440,1,Load_profile_3.csv,True
3,LOAD4,1,11.0,A,0.23,1,wye,1,0.95,Shape_4,Shape_4,1440,1,Load_profile_4.csv,True
4,LOAD5,1,12.0,A,0.23,1,wye,1,0.95,Shape_5,Shape_5,1440,1,Load_profile_5.csv,True
5,LOAD6,1,13.0,B,0.23,1,wye,1,0.95,Shape_6,Shape_6,1440,1,Load_profile_6.csv,True
6,LOAD7,1,21.0,B,0.23,1,wye,1,0.95,Shape_7,Shape_7,1440,1,Load_profile_7.csv,True
7,LOAD8,1,25.0,C,0.23,1,wye,1,0.95,Shape_8,Shape_8,1440,1,Load_profile_8.csv,True
8,LOAD9,1,26.0,A,0.23,1,wye,1,0.95,Shape_9,Shape_9,1440,1,Load_profile_9.csv,True
9,LOAD10,1,29.2,B,0.23,1,wye,1,0.95,Shape_10,Shape_10,1440,1,Load_profile_10.csv,True


In [5]:
ts_data = {}
for _, row in merged_load.iterrows():
    # print(row["File"])
    file_path = os.path.join(data_dir, 'Load Profiles', row["File"])
    try:
        profile = pd.read_csv(file_path)
        ts_data[row["Name_x"]] = profile["mult"].values * 1e-3 # or .to_numpy()
    except:
        profile = pd.read_csv(file_path, sep=';')
        ts_data[row["Name_x"]] = profile["mult"].values * 1e-3 # or .to_numpy()

profile_df = pd.DataFrame(ts_data)
profile_df.head(10)

,LOAD1,LOAD2,LOAD3,LOAD4,LOAD5,LOAD6,LOAD7,LOAD8,LOAD9,LOAD10,...,LOAD46,LOAD47,LOAD48,LOAD49,LOAD50,LOAD51,LOAD52,LOAD53,LOAD54,LOAD55
0,0.000036,0.000044,0.000054,0.000048,0.000035,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000062,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.000048,0.000055
1,0.000036,0.000044,0.000054,0.000048,0.000035,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000062,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.000048,0.000055
2,0.000036,0.000044,0.000054,0.000048,0.000035,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000122,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.000057,0.000055
3,0.000036,0.000044,0.000054,0.000227,0.000035,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000162,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.000048,0.000055
4,0.000036,0.000044,0.000054,0.000227,0.000035,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000222,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.000048,0.000055
5,0.000036,0.000044,0.000054,0.000227,0.000035,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000162,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.000048,0.000055
6,0.000036,0.000044,0.000054,0.000227,0.000147,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000162,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.000048,0.000055
7,0.000036,0.000044,0.000054,0.000227,0.000147,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000162,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.000148,0.000055
8,0.000036,0.000044,0.000054,0.000227,0.000147,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000162,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.000148,0.000055
9,0.000036,0.000044,0.000054,0.000227,0.000279,0.000052,0.000055,0.000043,0.000059,0.000057,...,0.000259,0.000045,0.000046,0.000054,0.000052,0.000053,0.000052,0.000047,0.002331,0.000055


In [6]:
import pandapower.control as ppc
from pandapower.timeseries.data_sources.frame_data import DFData

filtered_loads = merged_load[merged_load["Name_x"].isin(net48.asymmetric_load.name.values)]

ds = DFData(profile_df)

for load_name in net48.asymmetric_load.name:
    # get the index of the corresponding load
    idx = net48.asymmetric_load[net48.asymmetric_load.name == load_name].index[0]

    # find phase from the merged metadata
    phase = filtered_loads[filtered_loads["Name_x"] == load_name]["phases"].values[0].lower()

    ppc.ConstControl(
        net48, element_index=idx,
        element="asymmetric_load",
        element_name=f"CTRL_{net48.asymmetric_load.name.at[idx]}",
        variable=f"p_{phase}_mw",
        profile_name=load_name, data_source=ds
    )
net48

This pandapower network includes the following parameter tables:
   - bus (117 elements)
   - asymmetric_load (48 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (115 elements)
   - trafo (1 element)
   - controller (49 elements)
 and the following results tables:
   - res_bus_3ph (117 elements)
   - res_line_3ph (115 elements)
   - res_trafo_3ph (1 element)
   - res_ext_grid_3ph (1 element)
   - res_shunt_3ph (1 element)

In [7]:
output_dir = os.path.join(main_dir, "timeseries_results")

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [8]:
from pandapower.timeseries import run_timeseries
from pandapower.timeseries import OutputWriter

net48 = from_json(os.path.join(main_dir, 'json_networks', '48_loads_network.json'))
net48

# Create and attach output writer to net48
# ow48 = OutputWriter(net48, time_steps=range(profile_df.shape[0]), output_path=output_dir)
# # Log desired variables
# ow48.log_variable('res_trafo_3ph', 'loading_percent', index=net48.trafo.index[0])

# ow48.log_variable('res_bus_3ph', 'vm_b_pu', index=[32, 26]) # LOAD2 and LOAD23
# ow48.log_variable('res_bus_3ph', 'vm_a_pu', index=[14]) # LOAD55

# run_timeseries(net48, time_steps=range(profile_df.shape[0]), run=pp.runpp_3ph, continue_on_divergence=False)

This pandapower network includes the following parameter tables:
   - bus (117 elements)
   - asymmetric_load (48 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (115 elements)
   - trafo (1 element)
   - controller (1 element)
 and the following results tables:
   - res_bus_3ph (117 elements)
   - res_line_3ph (115 elements)
   - res_trafo_3ph (1 element)
   - res_ext_grid_3ph (1 element)
   - res_shunt_3ph (1 element)

In [9]:
net = from_json(os.path.join(main_dir, 'json_networks', 'no_load_network.json'))

# Create Loads
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
# print(loads_df.head(10))

net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
load_map = {}
for _, row in loads_df.iterrows():
    # print(_, '->', row)
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0
    if (bus_id in net.bus.name) & (row['phases'] in {'A', 'B', 'C'}):
        if row['phases']=='A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases']=='B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else: # row['phases']=='C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))
        load = pp.create_asymmetric_load(
            net, bus=net.bus.index[net.bus.name==bus_id][0],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
        load_map[load] = row['Yearly']
net

This pandapower network includes the following parameter tables:
   - bus (117 elements)
   - asymmetric_load (55 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (115 elements)
   - trafo (1 element)
   - controller (1 element)
 and the following results tables:
   - res_bus_3ph (117 elements)
   - res_line_3ph (115 elements)
   - res_trafo_3ph (1 element)
   - res_ext_grid_3ph (1 element)
   - res_shunt_3ph (1 element)

In [10]:
import pandapower.control as ppc
from pandapower.timeseries.data_sources.frame_data import DFData

ds = DFData(profile_df)

for load_name in net.asymmetric_load.name:
    # get the index of the corresponding load
    idx = net.asymmetric_load[net.asymmetric_load.name == load_name].index[0]

    # find phase from the merged metadata
    phase = merged_load[merged_load["Name_x"] == load_name]["phases"].values[0].lower()

    ppc.ConstControl(
        net, element_index=idx,
        element="asymmetric_load",
        element_name=f"CTRL_{net.asymmetric_load.name.at[idx]}",
        variable=f"p_{phase}_mw",
        profile_name=load_name, data_source=ds
    )
net

This pandapower network includes the following parameter tables:
   - bus (117 elements)
   - asymmetric_load (55 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (115 elements)
   - trafo (1 element)
   - controller (56 elements)
 and the following results tables:
   - res_bus_3ph (117 elements)
   - res_line_3ph (115 elements)
   - res_trafo_3ph (1 element)
   - res_ext_grid_3ph (1 element)
   - res_shunt_3ph (1 element)

In [11]:
# Create and attach output writer to net48
ow = OutputWriter(net, time_steps=range(profile_df.shape[0]), output_path=output_dir)
# Log desired variables
ow.log_variable('res_trafo_3ph', 'loading_percent', index=net.trafo.index[0])

ow.log_variable('res_bus_3ph', 'vm_a_pu', index=net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_b_pu', index=net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_c_pu', index=net.bus.index)

ow.log_variable('res_line_3ph', 'loading_a_percent', index=net.line.index)
ow.log_variable('res_line_3ph', 'loading_b_percent', index=net.line.index)
ow.log_variable('res_line_3ph', 'loading_c_percent', index=net.line.index)

run_timeseries(net, time_steps=range(profile_df.shape[0]), run=pp.runpp_3ph, run_control=True, continue_on_divergence=True)

  0%|          | 1/1440 [00:00<02:25,  9.86it/s]

KeyError: 1

In [ ]:
vm_a_pu = pd.read_pickle(os.path.join(output_dir, 'res_bus_3ph', 'vm_a_pu.p'))
vm_b_pu = pd.read_pickle(os.path.join(output_dir, 'res_bus_3ph', 'vm_b_pu.p'))
vm_c_pu = pd.read_pickle(os.path.join(output_dir, 'res_bus_3ph', 'vm_c_pu.p'))

trafo_loading = pd.read_pickle(os.path.join(output_dir, 'res_trafo_3ph', 'loading_percent.p'))

vm_a_pu.shape, vm_b_pu.shape, vm_c_pu.shape, trafo_loading.shape

In [ ]:
vm_a_pu.to_csv(os.path.join(output_dir, 'vm_a_pu.csv'))
vm_b_pu.to_csv(os.path.join(output_dir, 'vm_b_pu.csv'))
vm_c_pu.to_csv(os.path.join(output_dir, 'vm_c_pu.csv'))
net.res_bus_3ph.to_csv(os.path.join(output_dir, 'net.res_bus_3ph.csv'))
trafo_loading.to_csv(os.path.join(output_dir, 'trafo_loading_percent.csv'))

vm_a_pu.head(20)

In [ ]:
trafo_loading.describe()